# Beam Sensor Model Implementation in PyBullet

## Assignment Objectives:
- Implement Beam Sensor Model in PyBullet simulation
- Enable robot movement in cluttered environment with collision detection
- Detect range distance to collisions
- Semi-circular obstacle boundary at 5 meters
- Maximum scanning range: 8 meters
- Trajectory revision when collision detected within 0.2 meters

## Implementation Features:
- 5-beam sensor system at angles: [-40°, -20°, 0°, 20°, 40°]
- Real-time visualization of beams and distances
- Obstacle avoidance algorithm
- Noise simulation in sensor readings

In [1]:
# Import required libraries
import pybullet as p
import pybullet_data
import numpy as np
import time
import sys

print("Libraries imported successfully!")
print(f"PyBullet version: {p.getAPIVersion()}")

Libraries imported successfully!
PyBullet version: 202010061


## Robot Creation Function

Creates a simple box-shaped robot with collision and visual components.

In [2]:
def create_box_robot(position=[0, 0, 0.1], half_extents=[0.2, 0.2, 0.1]):
    """
    Create a simple box robot
    
    Args:
        position: [x, y, z] starting position
        half_extents: [x, y, z] half dimensions of the box
    
    Returns:
        robot_id: PyBullet object ID or -1 if failed
    """
    try:
        collision_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=half_extents)
        visual_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=half_extents, rgbaColor=[1, 0, 0, 1])
        robot_id = p.createMultiBody(baseMass=1,
                                     baseCollisionShapeIndex=collision_shape,
                                     baseVisualShapeIndex=visual_shape,
                                     basePosition=position)
        print(f"Robot created successfully with ID: {robot_id}")
        return robot_id
    except Exception as e:
        print(f"Error creating box robot: {e}")
        return -1

## Obstacle Creation Function

Creates a semi-circular arrangement of obstacles exactly 5 meters away from the robot's starting position.

In [3]:
def create_obstacles(radius=5.0, count=18, height=2.0):
    """
    Create semi-circular obstacles
    
    Args:
        radius: Distance from origin (5.0 meters as per requirement)
        count: Number of obstacles
        height: Height of obstacles
    """
    try:
        colors = [[1, 0, 0, 1], [0, 1, 0, 1], [0, 0, 1, 1], [1, 1, 0, 1]]
        half_extents = [0.15, 0.15, height / 2.0]
        obstacle_ids = []
        
        for i in range(count):
            angle = np.pi * i / (count - 1)  # Semi-circle from 0 to π
            x = radius * np.cos(angle)
            y = radius * np.sin(angle)
            color = colors[i % len(colors)]
            
            col_shape = p.createCollisionShape(p.GEOM_BOX, halfExtents=half_extents)
            vis_shape = p.createVisualShape(p.GEOM_BOX, halfExtents=half_extents, rgbaColor=color)
            obstacle_id = p.createMultiBody(baseMass=0,
                              baseCollisionShapeIndex=col_shape,
                              baseVisualShapeIndex=vis_shape,
                              basePosition=[x, y, half_extents[2]])
            obstacle_ids.append(obstacle_id)
            
        print(f"Created {count} obstacles in semi-circular arrangement at {radius}m radius")
        return obstacle_ids
    except Exception as e:
        print(f"Error creating obstacles: {e}")
        return []

## Robot Movement Function

Controls robot movement including forward motion and turning for obstacle avoidance.

In [4]:
def move_robot(robot_id, velocity=0.15, turn=False):
    """
    Move the robot forward or turn
    
    Args:
        robot_id: PyBullet robot object ID
        velocity: Forward movement speed
        turn: Boolean flag for turning
    """
    try:
        pos, orn = p.getBasePositionAndOrientation(robot_id)
        yaw = p.getEulerFromQuaternion(orn)[2]
        
        if turn:
            yaw += np.pi / 3  # 60 degree turn
            print(f"Robot turning: new yaw = {np.degrees(yaw):.1f}°")
        
        dx = velocity * np.cos(yaw)
        dy = velocity * np.sin(yaw)
        new_pos = [pos[0] + dx, pos[1] + dy, pos[2]]
        new_orn = p.getQuaternionFromEuler([0, 0, yaw])
        
        p.resetBasePositionAndOrientation(robot_id, new_pos, new_orn)
    except Exception as e:
        print(f"Error moving robot: {e}")

## Beam Sensor Model Implementation

Core beam sensor system that casts 5 rays at specific angles to detect obstacles and measure distances.

In [5]:
def cast_multiple_beams(robot_id, scan_range=8.0, noise_std=0.02):
    """
    Cast 5 beams at angles [-40°, -20°, 0°, 20°, 40°]
    
    Args:
        robot_id: PyBullet robot object ID
        scan_range: Maximum scanning range (8.0 meters as per requirement)
        noise_std: Standard deviation for sensor noise
    
    Returns:
        pos: Current robot position
        beam_data: List of (distance, beam_dir, beam_end) tuples
    """
    try:
        pos, orn = p.getBasePositionAndOrientation(robot_id)
        yaw = p.getEulerFromQuaternion(orn)[2]

        beam_angles_deg = [-40, -20, 0, 20, 40]  # 5-beam configuration
        beam_data = []

        for angle_deg in beam_angles_deg:
            angle_rad = yaw + np.deg2rad(angle_deg)
            beam_dir = [np.cos(angle_rad), np.sin(angle_rad), 0]
            beam_end = [pos[0] + scan_range * beam_dir[0],
                        pos[1] + scan_range * beam_dir[1],
                        pos[2]]

            # Ray casting for collision detection
            ray_result = p.rayTest(pos, beam_end)[0]
            hit_fraction = ray_result[2]
            distance = hit_fraction * scan_range

            # Add sensor noise
            noisy_distance = distance + np.random.normal(0, noise_std)
            noisy_distance = np.clip(noisy_distance, 0.0, scan_range)

            beam_data.append((noisy_distance, beam_dir, beam_end))

        return pos, beam_data
    except Exception as e:
        print(f"Error casting beams: {e}")
        return [0, 0, 0], []

## Simulation Setup

Initialize PyBullet environment, create robot and obstacles.

In [6]:

# Initialize PyBullet simulation
try:
    physicsClient = p.connect(p.GUI)
    if physicsClient < 0:
        print("CRITICAL ERROR: Could not connect to PyBullet.")
        sys.exit(1)
    
    print(f"PyBullet connected successfully. Client ID: {physicsClient}")
    
    # Set up physics environment
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    p.setGravity(0, 0, -9.8)
    
    # Load ground plane
    planeId = p.loadURDF("plane.urdf")
    if planeId < 0:
        print("CRITICAL ERROR: Could not load plane.")
        if p.isConnected(): p.disconnect()
        sys.exit(1)
    
    print(f"Ground plane loaded with ID: {planeId}")
    
    # Create robot
    robot_id = create_box_robot()
    if robot_id < 0:
        print("CRITICAL ERROR: Could not create robot.")
        if p.isConnected(): p.disconnect()
        sys.exit(1)
    
    # Create obstacles
    obstacle_ids = create_obstacles()
    
    # Set up camera view
    p.resetDebugVisualizerCamera(
        cameraDistance=8,
        cameraYaw=45,
        cameraPitch=-30,
        cameraTargetPosition=[0, 0, 0]
    )
    
    print("Simulation environment set up successfully!")
    
except Exception as e:
    print(f"Error setting up simulation: {e}")

PyBullet connected successfully. Client ID: 0
Ground plane loaded with ID: 0
Robot created successfully with ID: 1
Created 18 obstacles in semi-circular arrangement at 5.0m radius
Simulation environment set up successfully!


## Simulation Parameters

Define key parameters according to assignment requirements.

In [7]:
# Simulation parameters (as per assignment requirements)
SCAN_RANGE = 8.0           # Maximum scanning range: 8 meters
STOP_DISTANCE = 0.2        # Collision threshold: 0.2 meters
SENSOR_NOISE_STD = 0.02    # Sensor noise standard deviation
ROBOT_VELOCITY = 0.15      # Robot movement velocity
SIMULATION_STEP = 0.05     # Time step for simulation

print("Simulation Parameters:")
print(f"- Scan Range: {SCAN_RANGE} meters")
print(f"- Stop Distance: {STOP_DISTANCE} meters")
print(f"- Sensor Noise: {SENSOR_NOISE_STD}")
print(f"- Robot Velocity: {ROBOT_VELOCITY} m/step")
print(f"- Simulation Step: {SIMULATION_STEP} seconds")

Simulation Parameters:
- Scan Range: 8.0 meters
- Stop Distance: 0.2 meters
- Sensor Noise: 0.02
- Robot Velocity: 0.15 m/step
- Simulation Step: 0.05 seconds


## Main Simulation Loop

Execute the main simulation with beam sensing and obstacle avoidance.

**Instructions:**
- Run this cell to start the simulation
- The robot will move forward until it detects an obstacle within 0.2 meters
- When obstacle detected, robot will turn 60° and continue
- Press Ctrl+C in the output to stop the simulation
- Beam colors: Red = Danger zone (<0.2m), Green = Safe zone
- Distance values are displayed next to each beam

In [8]:
# Main simulation loop
try:
    step_count = 0
    collision_count = 0
    
    print("Starting simulation...")
    print("Press Ctrl+C to stop the simulation")
    
    while True:
        p.stepSimulation()
        p.removeAllUserDebugItems()  # Clear previous visualizations
        
        # Cast 5 beams and get sensor data
        pos, beam_data = cast_multiple_beams(robot_id, SCAN_RANGE, SENSOR_NOISE_STD)
        
        min_distance = SCAN_RANGE
        
        # Visualize beams and distances
        for i, (distance, beam_dir, beam_end) in enumerate(beam_data):
            # Color coding: Red for danger zone, Green for safe zone
            color = [1, 0, 0] if distance < STOP_DISTANCE else [0, 1, 0]
            
            # Draw beam line
            p.addUserDebugLine(pos, beam_end, color, lineWidth=2, lifeTime=0.1)
            
            # Display distance text
            label_pos = [pos[0] + 0.2 * beam_dir[0],
                         pos[1] + 0.2 * beam_dir[1],
                         pos[2] + 0.1]
            p.addUserDebugText(f"{distance:.2f}m", label_pos,
                               textColorRGB=[0, 0, 0],
                               textSize=0.7,
                               lifeTime=0.1)
            
            min_distance = min(min_distance, distance)
        
        # Movement decision based on sensor readings
        if min_distance < STOP_DISTANCE:
            move_robot(robot_id, velocity=ROBOT_VELOCITY, turn=True)
            collision_count += 1
            print(f"Step {step_count}: Obstacle detected at {min_distance:.3f}m - TURNING (#{collision_count})")
        else:
            move_robot(robot_id, velocity=ROBOT_VELOCITY)
            if step_count % 100 == 0:  # Print status every 100 steps
                print(f"Step {step_count}: Moving forward - Min distance: {min_distance:.3f}m")
        
        step_count += 1
        time.sleep(SIMULATION_STEP)

except KeyboardInterrupt:
    print(f"\nSimulation interrupted by user after {step_count} steps.")
    print(f"Total obstacle avoidance maneuvers: {collision_count}")
except p.error as e:
    print(f"PyBullet Error: {e}")
except Exception as e:
    print(f"Unexpected Error: {e}")
finally:
    if p.isConnected():
        p.disconnect()
        print("PyBullet disconnected cleanly.")
    else:
        print("PyBullet was not connected or already disconnected.")

Starting simulation...
Press Ctrl+C to stop the simulation
Step 0: Moving forward - Min distance: 4.823m
Robot turning: new yaw = 60.0°
Step 31: Obstacle detected at 0.190m - TURNING (#1)
Robot turning: new yaw = 120.8°
Step 36: Obstacle detected at 0.117m - TURNING (#2)
Robot turning: new yaw = 181.5°
Step 55: Obstacle detected at 0.099m - TURNING (#3)
Step 100: Moving forward - Min distance: 0.575m
Robot turning: new yaw = -118.0°
Step 105: Obstacle detected at 0.180m - TURNING (#4)
Robot turning: new yaw = -58.0°
Step 106: Obstacle detected at 0.057m - TURNING (#5)
Step 200: Moving forward - Min distance: 7.977m
PyBullet Error: Not connected to physics server.
PyBullet was not connected or already disconnected.
